In [9]:
import os
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [12]:
df = pd.read_csv("../static_merged_data.csv")
x = df.drop(labels="status", axis=1)
y = df[["status"]]

def prepare_train_test_set(X, Y, num_classes):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, test_size=.2)
    train_labels = y_train.status.astype('category').cat.codes.to_numpy()
    test_labels = y_test.status.astype('category').cat.codes.to_numpy()
    train_labels = to_categorical(train_labels, num_classes)
    test_labels = to_categorical(test_labels, num_classes)
    X_train.to_numpy()
    X_test.to_numpy()
    return (X_train, X_test), (train_labels, test_labels)

In [13]:
(X_train, X_test), (train_labels, test_labels) = prepare_train_test_set(x,y,3)

In [17]:
from keras.models import Model
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Input, InputLayer
import tensorflow as tf
from keras import Sequential
from keras.layers import Activation 
model = Sequential([
    InputLayer(input_shape=(4,), name="input-layer"),
    Dense(64, activation="relu"),
    Dense(128, activation="relu"),
    Dense(128, activation="relu"),
    Dense(128, activation="relu"),
    Dense(16, activation="relu"),
    Dense(3, activation="softmax", name="output_layer")
], name="DCDC_type_classifier")

model.summary()

Model: "DCDC_type_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                320       
                                                                 
 dense_6 (Dense)             (None, 128)               8320      
                                                                 
 dense_7 (Dense)             (None, 128)               16512     
                                                                 
 dense_8 (Dense)             (None, 128)               16512     
                                                                 
 dense_9 (Dense)             (None, 16)                2064      
                                                                 
 output_layer (Dense)        (None, 3)                 51        
                                                                 
Total params: 43,779
Trainable params: 43,779


In [18]:
from keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.optimizers import Adam
model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=(model.name + ".h5"),
#     save_weights_only=False,
#     monitor='val_acc',
#     mode='max',
#     save_best_only=True)

history = model.fit(X_train, train_labels, 10, validation_split=0.1)

c:\Users\sandr\source\repos\Monitoring_Converter_ML\venv\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2288.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2288.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\sandr\source\repos\Monitoring_Converter_ML\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\sandr\source\repos\Monitoring_Converter_ML\venv\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\Users\sandr\source\repos\Monitoring_Converter_ML\venv\lib\site-packages\ipyk